##ETL Project (Cristalle Pronier and Hung Dang)

CMS Part D Prescriber Dataset (2017)

Overview

Increased scrutiny on the potential over-prescription of habit-forming drugs such as opioids has led to more examination of reported data.  Our team choose to pull multiple healthcare datasets to create a database to examine this issue.    We choose to narrow our data to records for California only for this project due to the sheer volume of records.  This write up describes the dataset extracted and transformed by Cristalle.

This national dataset is published by the Centers for Medicare & Medicaid Services (CMS) and contains the names of all medical providers and their corresponding pharmaceutical prescriptions with related prescriber data who bill medical services under a CMS certified facility.  This dataset also contains the National Provider Identifier (NPI) which is a unique number assigned to each provider.  The second dataset in this project extracted and transformed by Hung also includes the NPI which will be a key link in joining the datasets prior to analysis.


In [1]:
#Import dependencies required for data extraction and transformation
import requests
import json
import pandas as pd
import numpy as np

##Extract

The original dataset contains over 25 million records and is publicly available at the following link: https://data.cms.gov/Medicare-Part-D/Medicare-Provider-Utilization-and-Payment-Data-201/77gb-8z53.  To pull data the CMS API (run by Socrata) was used with a filter query to return only California records using the two-digit postal abbreviation CA.  The API documentation for developers was consulted to figure out the API url to pull only California records (https://data.cms.gov/resource/bwz9-yuau.json?nppes_provider_state=CA).  

Initially, only 1,000 California records were being returned from the API call which is much lower than the expected amount of records.  To troubleshoot this issue, I contacted the data technical assistance team and searched the documentation.  I found in the documentation (team also emailed me the following day to confirm what I figured out) that there is a default limit of 1,000 records.  This could be changed to the maximum of 50,000 with an additional limit defined on the API url (documentation: https://dev.socrata.com/docs/paging.html).  The API url was configured to pull the maximum allowed records from California cases (https://data.cms.gov/resource/bwz9-yuau.json?nppes_provider_state=CA&$limit=50000).  Data were returned in JSON format.


In [2]:
data = requests.get("https://data.cms.gov/resource/bwz9-yuau.json?nppes_provider_state=CA&$limit=50000").json()

In [3]:
#Review first record of JSON data returned
data[0]

{'npi': '1861502288',
 'nppes_provider_last_org_name': 'SEYEDKAZEMI',
 'nppes_provider_first_name': 'MAHSHID',
 'nppes_provider_city': 'LOS ANGELES',
 'nppes_provider_state': 'CA',
 'specialty_desc': 'Pharmacist',
 'description_flag': 'T',
 'drug_name': 'CLONIDINE HCL',
 'generic_name': 'CLONIDINE HCL',
 'total_claim_count': '30',
 'total_30_day_fill_count': '36',
 'total_day_supply': '979',
 'total_drug_cost': '207.07',
 'bene_count_ge65_suppress_flag': '*',
 'ge65_suppress_flag': '*'}

In [4]:
#Confirm the API call returned the expected number of 50,000 maximum records for California.  
len(data)

50000

In [5]:
df = pd.DataFrame.from_dict(data, orient='columns')

In [6]:
df.head()

,bene_count,bene_count_ge65,bene_count_ge65_suppress_flag,description_flag,drug_name,ge65_suppress_flag,generic_name,npi,nppes_provider_city,nppes_provider_first_name,...,nppes_provider_state,specialty_desc,total_30_day_fill_count,total_30_day_fill_count_ge65,total_claim_count,total_claim_count_ge65,total_day_supply,total_day_supply_ge65,total_drug_cost,total_drug_cost_ge65
0,NaN,NaN,*,T,CLONIDINE HCL,*,CLONIDINE HCL,1861502288,LOS ANGELES,MAHSHID,...,CA,Pharmacist,36,NaN,30,NaN,979,NaN,207.07,NaN
1,NaN,NaN,*,S,EZETIMIBE,NaN,EZETIMIBE,1801848825,THOUSAND OAKS,MANOJ,...,CA,Internal Medicine,51,51,17,17,1530,1530,4043.47,4043.47
2,NaN,NaN,*,S,CLINDAMYCIN HCL,*,CLINDAMYCIN HCL,1174674436,WATSONVILLE,NICOLE,...,CA,Podiatry,15,NaN,15,NaN,190,NaN,251.73,NaN
3,NaN,NaN,#,S,CLONIDINE HCL,#,CLONIDINE HCL,1881950632,SACRAMENTO,AVA,...,CA,Family Practice,26,NaN,26,NaN,765,NaN,255.92,NaN
4,45,45,NaN,S,AMLODIPINE BESYLATE,NaN,AMLODIPINE BESYLATE,1043347925,LOS ANGELES,RICHARD,...,CA,Cardiology,454.5,454.5,264,264,13606,13606,2519.73,2519.73


In [7]:
df.count()

bene_count                       20474
bene_count_ge65                   7480
bene_count_ge65_suppress_flag    42520
description_flag                 50000
drug_name                        50000
ge65_suppress_flag               20527
generic_name                     50000
npi                              50000
nppes_provider_city              50000
nppes_provider_first_name        50000
nppes_provider_last_org_name     50000
nppes_provider_state             50000
specialty_desc                   50000
total_30_day_fill_count          50000
total_30_day_fill_count_ge65     29473
total_claim_count                50000
total_claim_count_ge65           29473
total_day_supply                 50000
total_day_supply_ge65            29473
total_drug_cost                  50000
total_drug_cost_ge65             29473
dtype: int64

In [8]:
#Review the data info and get an idea of null data in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
bene_count                       20474 non-null object
bene_count_ge65                  7480 non-null object
bene_count_ge65_suppress_flag    42520 non-null object
description_flag                 50000 non-null object
drug_name                        50000 non-null object
ge65_suppress_flag               20527 non-null object
generic_name                     50000 non-null object
npi                              50000 non-null object
nppes_provider_city              50000 non-null object
nppes_provider_first_name        50000 non-null object
nppes_provider_last_org_name     50000 non-null object
nppes_provider_state             50000 non-null object
specialty_desc                   50000 non-null object
total_30_day_fill_count          50000 non-null object
total_30_day_fill_count_ge65     29473 non-null object
total_claim_count                50000 non-null object
total_claim_

##Transform

The JSON data was transformed into a Pandas Dataframe prior to cleaning the data.  The dataset documentation was reviewed to better understand the data and to make informed decisions on data cleaning.  

Three columns of unnecessary data were dropped from the Dataframe: bene_count_ge65_suppress_flag, description_flag and ge65_suppress_flag.  These data referred to the “suppression” flags and reasons if they were flagged.  In this dataset, some of the data are suppressed meaning that it is not reported for one of several reasons.  The unreported data shows up as “NaN.” The most common reason for suppressed data is a low number reported, in the case of this dataset, any number reported between 1-10 will be suppressed and shown as NaN.  

Null values were left as they were originally reported as some columns have a substantial amount.  It was not appropriate to delete all records with a null in one or more column since there are so many and it was not appropriate to assign a value of zero because that could skew numeric analysis done with the data.  Instead, future analysis using these data in the constructed database should consider how to handle the null values on a case-by-case basis.


In [9]:
#Drop columns with data not needed for database and analysis
df = df.drop(columns=['bene_count_ge65_suppress_flag', 'description_flag', 'ge65_suppress_flag'])

In [18]:
#Confirm columns were dropped
df.head()

,bene_count,bene_count_ge65,drug_name,generic_name,nppes_provider_city,nppes_provider_first_name,nppes_provider_last_org_name,nppes_provider_state,specialty_desc,total_30_day_fill_count,total_30_day_fill_count_ge65,total_claim_count,total_claim_count_ge65,total_day_supply,total_day_supply_ge65,total_drug_cost,total_drug_cost_ge65
npi,,,,,,,,,,,,,,,,,
1861502288,NaN,NaN,CLONIDINE HCL,CLONIDINE HCL,LOS ANGELES,MAHSHID,SEYEDKAZEMI,CA,Pharmacist,36,NaN,30,NaN,979,NaN,207.07,NaN
1801848825,NaN,NaN,EZETIMIBE,EZETIMIBE,THOUSAND OAKS,MANOJ,ASWANI,CA,Internal Medicine,51,51,17,17,1530,1530,4043.47,4043.47
1174674436,NaN,NaN,CLINDAMYCIN HCL,CLINDAMYCIN HCL,WATSONVILLE,NICOLE,SURDOCK,CA,Podiatry,15,NaN,15,NaN,190,NaN,251.73,NaN
1881950632,NaN,NaN,CLONIDINE HCL,CLONIDINE HCL,SACRAMENTO,AVA,ASHER,CA,Family Practice,26,NaN,26,NaN,765,NaN,255.92,NaN
1043347925,45,45,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE,LOS ANGELES,RICHARD,GOLD,CA,Cardiology,454.5,454.5,264,264,13606,13606,2519.73,2519.73


In [19]:
#Set the National Provider Identifier as the column index
#df = df.set_index('npi', inplace=True)

In [21]:
df.to_csv('healthcare_prescriber_output.csv')

##Load

The cleaned dataset is loaded into a MySQL database.  MySQL was chosen because the existing tabular data structure would work best.  Loading Pandas Dataframes into MySQL is a straightforward process.  To load easily into MySQL the Pandas Dataframe is written into a csv file.  The csv file is loaded into the database in MySQL created for this project as a new table.  With both datasets loaded into MySQL they can be queried, joined and reviewed as needed.
